In [ ]:
import os
import numpy as np
import tensorflow as tf

In [ ]:
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
import cv2
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing import image

In [ ]:
## the kaggle zip file extract put the files in folders for each note type
# to simplify the below code, all files were copied out of their subdirectories into single "Images" folder. 
#Images folder is part of gitignore
counter = 0 #for testing/break purposes only
img_data_array=[]
class_name=[]
folder_name = "Images"
#This code based on https://towardsdatascience.com/loading-custom-image-dataset-for-deep-learning-models-part-1-d64fa7aaeca6
for file in os.listdir(os.path.join(folder_name)):
    counter +=1
    image_path= os.path.join(folder_name,  file)
    image= cv2.imread( image_path, cv2.IMREAD_GRAYSCALE)
    #image=np.array(image)  ????
#     print(image.shape)
    image = image.astype('float32')
    image /= 255 #this gets black and white to 1s and 0s 
    img_data_array.append(image)
    note_class = file[0:1]
#     print(note_class)
    class_name.append(note_class)
#    if (counter>5):
#        break;


In [ ]:
def create_dataset(notes_folder):
    class_name = []
    img_data_array = []
    
    for note_dir in os.listdir(notes_folder):
        
        for file in os.listdir(os.path.join(notes_folder, note_dir)):
            image = os.path.join(notes_folder, note_dir, file)
            image = cv2.imread(notes_folder, cv2.COLOR_BGR2RGB)
            #image = cv2.resize( image, (IMG_HEIGHT, IMG_WIDTH), interpolation = cv2.INTER_AREA)
            #image = np.array(image)  # converts the image to a numpy array
            image = image.astype('float32')
            image /= 255
            img_data_array.append(image)
            class_name.append(note_dir)
            
    print(img_data_array[0])
    return img_data_array, class_name

In [ ]:
# Assigning a unique number to each class
note_classifier_dict = {key:value for value, key in enumerate(np.unique(class_name))}

In [ ]:
note_classifier_dict

In [ ]:
# This is y, converting the class names to a numeric value

target_val=  [note_classifier_dict[class_name[i]] for i in range(len(class_name))]

In [ ]:
# target_val

In [ ]:
# Examine the image array ... setting the threshold to 4096 or above avoids truncating the array

np.set_printoptions(threshold=4096)
img_data_array[0]

In [ ]:
#test to make sure our images are still good.
import matplotlib.pyplot as plt
plt.imshow(img_data_array[10], cmap=plt.cm.Greys)

In [ ]:
# flatten the 64x64 pixels in to a 1D array 
#similar to what was done in MNIST class example
num_dimensions = 64*64

#need to make this an np array so we can use reshape
x=np.array(img_data_array, np.float32)

reshape_test = x.reshape(x.shape[0],num_dimensions)

In [ ]:
reshape_test.shape

In [ ]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(reshape_test,target_val, random_state=42)

## Scaling and Normalization

In [ ]:
# Next, we normalize our training data to be between 0 and 1
# scaler = MinMaxScaler().fit(X_train)

# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
# X_train[0]

## One-Hot Encoding

In [ ]:
# Our Training and Testing labels are integer encoded from 0 to 4
y_train[:20]

In [ ]:

num_classes = 5
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_train[0]

## Building our Model


In [ ]:
# Create an empty sequential model
model = Sequential()

In [ ]:
# Add the first layer where the input dimensions are the 4096 pixel values
# Activation function ... starting with 'relu'
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))


### Add a second hidden layer with 100 densely connected nodes

In [ ]:
model.add(Dense(100, activation='relu'))

### Final output layer uses softmax activation function for logistic regression

In [ ]:
model.add(Dense(num_classes, activation='softmax'))

### Model Summary

In [ ]:
model.summary()

### Compile and Train (woof woof) our Model

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Train (fit) our model using the training data

In [ ]:
var = model.fit(
    X_train,
    y_train,
    epochs=10,
    shuffle=True,
    verbose=2
    )

In [ ]:
var.model

In [ ]:
# identify visualizations